In [1]:
import numpy as np

# Get inputs

In [2]:
# with open("test_input.txt") as f:
with open("input.txt") as f:
    inp = f.readlines()
    inp = [line.rstrip() for line in inp]

In [3]:
cleaned = []
for line in inp:
    state, coords = line.split()
    x,y,z = coords.split(',')
    x = x[2:].split('..')
    y = y[2:].split('..')
    z = z[2:].split('..')
    
    cleaned.append([state, [(int(x[0]), int(x[1])+1), (int(y[0]), int(y[1])+1), (int(z[0]), int(z[1])+1)]])

In [4]:
# cleaned

## Q1

In [5]:
reactor = np.full((101,101,101), 0,dtype=np.int8)

In [6]:
def run(reactor, steps):
    n = 50
    for state, coords in steps:
        x,y,z = coords
        x = np.add(x,n)
        y = np.add(y,n)
        z = np.add(z,n)

        state = 1 if 'on' in state else 0
        
        reactor[x[0]:x[1], y[0]:y[1], z[0]:z[1]] = state
    
    
    return reactor

In [7]:
print(f"Part 1: {np.sum(run(reactor, cleaned))}")

Part 1: 607657


## Q2

In [8]:
def check_intersect(cuboid_1, cuboid_2):
    """Check if there is any intersection along any dimension"""
    
    # 1minx <= 2maxx and 1maxx >= 2minx
    if not(cuboid_1[1] <= cuboid_2[2] and cuboid_1[2] >= cuboid_2[1]):
        return False
 
    if not(cuboid_1[3] <= cuboid_2[4] and cuboid_1[4] >= cuboid_2[3]):
        return False
 
    if not(cuboid_1[5] <= cuboid_2[6] and cuboid_1[6] >= cuboid_2[5]):
        return False

    return True

In [9]:
def get_intersection(cuboid_1, cuboid_2):
    """Get the intersection, including if the resulting state is on or off"""
    
    min_x = max(cuboid_1[1], cuboid_2[1])
    max_x = min(cuboid_1[2], cuboid_2[2])
 
    min_y = max(cuboid_1[3], cuboid_2[3])
    max_y = min(cuboid_1[4], cuboid_2[4])
 
    min_z = max(cuboid_1[5], cuboid_2[5])
    max_z = min(cuboid_1[6], cuboid_2[6])
 
    if cuboid_1[0] == cuboid_2[0]:
        state = 0 if cuboid_1[0] == 1 else 1
    elif cuboid_1[0] == 1 and cuboid_2[0] == 0:
        state = 1
    else:
        state = 0
 
    return (state, min_x, max_x, min_y, max_y, min_z, max_z)

In [10]:
def get_volume(cuboid):
    x = cuboid[2] - cuboid[1]
    y = cuboid[4] - cuboid[3]
    z = cuboid[6] - cuboid[5]
    
    return x * y * z

In [11]:
def get_vol(steps):
    cuboids = []
 
    for state, coords in steps:
        min_x, max_x = coords[0]
        min_y, max_y = coords[1]
        min_z, max_z = coords[2]
 
        current_cuboid = (1 if state == 'on' else 0,
                min_x, max_x,
                min_y, max_y,
                min_z, max_z,
                )
    
        intersections = []
 
        for cuboid in cuboids:
            if check_intersect(current_cuboid, cuboid):
                intersection = get_intersection(current_cuboid, cuboid)
                intersections.append(intersection)
 
        for intersection in intersections:
            cuboids.append(intersection)
 
        if state == 'on':
            cuboids.append(current_cuboid)
 
    on = 0
 
    for cuboid in cuboids:
        if cuboid[0] == 1:
            on += get_volume(cuboid)
        else:
            on -= get_volume(cuboid)
 
    return on

In [12]:
print(f"Part 2: {get_vol(cleaned)}")

Part 2: 1187742789778677
